In [2]:
import tensorflow as tf
import logging, sys
logging.disable(sys.maxsize)

print(tf.__version__)


DIMA = 32
DIMB = 128
DIMM = 32
DIMN = 513
DIMK = 128

class Model(tf.Module):
    @tf.function(input_signature=[tf.TensorSpec(shape=[DIMA, DIMB, DIMM, DIMK], dtype=tf.float32, name=None), 
                                  tf.TensorSpec(shape=[DIMA, DIMB, DIMK, DIMN], dtype=tf.float32, name=None)])
    def func1(self, x, y):
        return tf.matmul(x,y)

model = Model()

converter = tf.lite.TFLiteConverter.from_concrete_functions([model.func1.get_concrete_function()], model)
tflite_model = converter.convert() 

with open('genBatchGEMM_float32.tflite', 'wb') as f:
    f.write(tflite_model)

interpreter = tf.lite.Interpreter(model_content=tflite_model)
signatures = interpreter.get_signature_list()
print(signatures)

2.13.0
{'serving_default': {'inputs': ['x', 'y'], 'outputs': ['output_0']}}


In [3]:
#Integer Float Fallback Quantization
#?Large File?
converter = tf.lite.TFLiteConverter.from_concrete_functions([model.func1.get_concrete_function()], model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
def representative_dataset():
    for i in range(10): # create fake data
        fake_data_x = tf.random.normal([DIMA, DIMB, DIMM, DIMK])
        fake_data_y = tf.random.normal([DIMA, DIMB, DIMK, DIMN])
        yield [fake_data_y, fake_data_x]
converter.representative_dataset = representative_dataset

tflite_model_integer_fallback = converter.convert()

with open('genBatchGEMM_integer_fallback.tflite', 'wb') as f:
  f.write(tflite_model_integer_fallback)

In [4]:
#integer-only quantization
#?Large File?
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_integer_only = converter.convert()

with open('genBatchGEMM_integer_only.tflite', 'wb') as f:
  f.write(tflite_model_integer_only)